<h3> How to Write Advance SQL Queries | Consecutive Empty Seats | SQL Interview Questions </h3>

In [0]:
%sql
create table bms (seat_no int ,is_empty varchar(10));
insert into bms values
(1,'N')
,(2,'Y')
,(3,'N')
,(4,'Y')
,(5,'Y')
,(6,'Y')
,(7,'N')
,(8,'Y')
,(9,'Y')
,(10,'Y')
,(11,'Y')
,(12,'N')
,(13,'Y')
,(14,'Y');

num_affected_rows,num_inserted_rows
14,14


In [0]:
%sql
with cte as(
select *,lag(is_empty,1,'N')over(order by seat_no asc) as lag1,
lag(is_empty,2,'N')over(order by seat_no asc) as lag2 from bms
)
select distinct bms.seat_no from bms 
inner join(
select seat_no from cte where is_empty='Y' AND LAG1='Y' AND LAG2='Y'
)a on (bms.seat_no<=a.seat_no and bms.seat_no>=a.seat_no-2)


seat_no
6
5
9
4
8
10
11


In [0]:
%sql
with cte as(
select *,seat_no-rn as diff from(
select *,row_number()over(order by seat_no)rn from bms where is_empty='Y'))

select seat_no from cte where diff in(
select diff from (
select diff,count(*) from cte
group by diff
having count(*)>2)a
)

seat_no
4
5
6
8
9
10
11


<h3> solution in pyspark <\h3>

In [0]:
from pyspark.sql.functions import lag,row_number,col
from pyspark.sql.window import Window

In [0]:
df_b=spark.sql("select * from bms")
df_b.show()

+-------+--------+
|seat_no|is_empty|
+-------+--------+
|      1|       N|
|      2|       Y|
|      3|       N|
|      4|       Y|
|      5|       Y|
|      6|       Y|
|      7|       N|
|      8|       Y|
|      9|       Y|
|     10|       Y|
|     11|       Y|
|     12|       N|
|     13|       Y|
|     14|       Y|
+-------+--------+



In [0]:
df_a=df_b.withColumn("lag1",lag("is_empty",1).over(Window.orderBy("seat_no"))).\
    withColumn("lag2",lag("is_empty",2).over(Window.orderBy("seat_no"))).\
        filter((col("is_empty")=="Y")&(col("lag1")=="Y")&(col("lag2")=="Y")).select(col("seat_no").alias("st_no"))
df_b.join(df_a,(df_b.seat_no<=df_a.st_no)&(df_b.seat_no>=df_a.st_no-2),"inner")\
    .select("seat_no").distinct().orderBy("seat_no").show()


+-------+
|seat_no|
+-------+
|      4|
|      5|
|      6|
|      8|
|      9|
|     10|
|     11|
+-------+



In [0]:
df_a=df_b.filter(col("is_empty")=="Y").withColumn("rnk",row_number().over(Window.orderBy("seat_no")))
df_a1=df_a.withColumn("diff",col("seat_no")-col("rnk"))
df_a1.groupBy("diff").count().filter(col("count")>2).select("diff")
df_a1.join(df,"diff","inner").select("seat_no").show()

+-------+
|seat_no|
+-------+
|      4|
|      5|
|      6|
|      8|
|      9|
|     10|
|     11|
+-------+

